# 🎉 User Rating Predictions using Matrix Factorization for the MovieLens Dataset 🎉

TeamMHL: Emmanouil Chatzakis, Hind El-Bouchrifi, Lluka Stojollari (emmanouil.chatzakis@epfl.ch, hind.el-bouchrifi@epfl.ch, lluka.stojollari@epfl.ch)

This notebook implements an optimized Matrix Factorization recommendation system for the MovieLens dataset.

November 2023, Distributed Information Systems, Project 2, EPFL

## Imports and Libraries 🚨

Here, we import all the needed libraries. Make sure to install all the frameworks and libraries used below.

In [ ]:
import torch
import time
import os

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import torch.optim as optim

from collections import defaultdict

from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau

from tqdm import tqdm

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

seed_value = 42
np.random.seed(seed_value)

## Object and Dataset definitions 💡

Here we define the Matrix Factorization modules and datasets, as well as helper functions to be used later.

In [ ]:
# Define the movielens dataser representation
class MovieRatingDataset(Dataset):
    """Dataset representing a movie for Matrix Factorization"""
    def __init__(self, dataframe, movie_to_genre_vector):
        self.dataframe = dataframe
        self.movie_to_genre_vector = movie_to_genre_vector
        self.x_user_movie = list(zip(self.dataframe.userId.values, self.dataframe.movieId.values))
        self.y_rating = self.dataframe.rating.values

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        movie_id = self.x_user_movie[idx][1]
        genre_vector = self.movie_to_genre_vector[movie_id.item()]

        return (
            self.x_user_movie[idx],
            self.y_rating[idx],
            genre_vector,
        ) 

In [ ]:
# Matrix Factorization modules and functions

def sigmoid_range(x, low, high):
    """Sigmoid function with range (low, high)"""
    return torch.sigmoid(x) * (high - low) + low


class MatrixFactorization(nn.Module):
    """Matrix Factorization module class"""
    def __init__(self, n_users, n_movies, n_factors, n_genre):
        super(MatrixFactorization, self).__init__()

        self.user_factors = nn.Embedding(n_users, n_factors)
        self.movie_factors = nn.Embedding(n_movies, n_factors)
        self.genre_factors = nn.Linear(n_genre, n_factors)  # as Linear

        self.user_bias = nn.Parameter(torch.zeros(n_users).normal_(0, 0.01))
        self.movie_bias = nn.Parameter(torch.zeros(n_movies).normal_(0, 0.01))

        self.offset = nn.Parameter(torch.zeros(1).normal_(0, 0.01))

        self.user_factors.weight.data.uniform_(0.0, 0.05)
        self.movie_factors.weight.data.uniform_(0.0, 0.05)
        self.genre_factors.weight.data.uniform_(0.0, 0.05)

    def forward(self, user, item, genre_vector):
        user_emb = self.user_factors(user)
        item_emb = self.movie_factors(item)

        genre_emb = self.genre_factors(genre_vector)

        item_emb += genre_emb

        element_product = (user_emb * item_emb).sum(1)

        user_b = self.user_bias[user]
        item_b = self.movie_bias[item]

        element_product += user_b + item_b + self.offset

        return sigmoid_range(element_product, 0, 5.5)

## Section 1: Data Loading and Preprocessing 📈

In this section we load MovieLens, preprocess and merge the data.

### Basic Data Loading and Processing

Here we load the raw MovieLens data.

In [ ]:
# Get all the data paths
current_directory = os.getcwd()
general_path = "/kaggle/input/dis-project-2-recommender-systems"

movies_path = os.path.join(general_path, "movies.csv")
links_path = os.path.join(general_path, "links.csv")
tags_path = os.path.join(general_path, "tags.csv")

In [ ]:
# Training and test ratings sets
training_path = os.path.join(general_path, "train_ratings.csv")
test_path = os.path.join(general_path, "test_set_no_ratings.csv")

In [ ]:
# Load the movie and link data into dataframes
movies = pd.read_csv(movies_path)
links = pd.read_csv(links_path)

movies = pd.concat([movies, links], axis=1)  # concatenate the two infromation dataframes for the movies
movies = movies.loc[:, ~movies.columns.duplicated()]  # remove the duplicate columns

movies.head()

In [ ]:
# Load and process the tag data
tags = pd.read_csv(tags_path)
tags.rename(columns={"userId": "user_tag_id"}, inplace=True)

grouped_tags = tags.groupby(["user_tag_id", "movieId"]).agg(list).reset_index()
grouped_tags.drop(columns="timestamp", inplace=True)
grouped_tags.head(1)

In [ ]:
# Prepare the training dataset
train = pd.read_csv(training_path)
train.drop(columns="timestamp", inplace=True)
test = pd.read_csv(test_path)

test.drop(columns="Id", inplace=True)
test["rating"] = -1

frames = [train, test]
ratings = pd.concat(frames)

merged_train = ratings.merge(movies, on="movieId", how="left")

# Prepare the testing dataset
test = pd.read_csv(test_path)

# Add the corresponding tags for each user-movieid combination
merged_full = pd.merge(
    merged_train,
    grouped_tags,
    how="left",
    left_on=["userId", "movieId"],
    right_on=["user_tag_id", "movieId"],
)

# Minor modifications to the merged dataframe
merged_full.drop(columns="user_tag_id", inplace=True)
merged_full["rating_count_per_user"] = merged_full.groupby("userId")["rating"].transform("count")
merged_full.drop(columns=["genres"], inplace=True)

merged_full.head()

### Genre Incorporation

Now, we include the available genres per movie to our data.

In [ ]:
# Load the full_movielens_geners
full_movies_path = "/kaggle/input/genres/full_movielens_genres.csv"
full_movies = pd.read_csv(full_movies_path)
full_movies = full_movies.groupby("imdbId")["genres"].first().reset_index()

merged_full = pd.merge(merged_full, full_movies, on="imdbId", how="left")

print(f"Number of unique user IDs: {merged_full['userId'].unique().size}")
print(f"Number of unique movie IDs: {merged_full['movieId'].unique().size}")

merged_full["rating_count_per_user"].describe()

### Calculate the average rating per user

In [ ]:
average_ratings_per_user = merged_full["rating_count_per_user"].mean()
print(f"Average number of ratings per user: {average_ratings_per_user:.2f}")

## Section 2: Implementing Matrix Factorization 🪛

In this section we prepare the data for Matrix Factorization, and then we implement and train the Matrix Factorization model.


### Prepare the data for the model

In [ ]:
# Data preparation for Matrix Factorization
test.drop(columns="Id", inplace=True)
test["rating"] = -1

frames = [train, test]
ratings = pd.concat(frames)  # concatenate train and test in order to create the matrix later
ratings = ratings.merge(movies, on="movieId", how="left")  # merge with movies
ratings.drop(columns=["title", "imdbId", "tmdbId"], inplace=True)

ratings.head()

In [ ]:
no_genres_rows = ratings[ratings["genres"] == "(no genres listed)"]

# Iterate through rows and update 'genres' based on 'merged_full'
for index, row in no_genres_rows.iterrows():
    movie_id = row["movieId"]
    corresponding_genre = merged_full.loc[merged_full["movieId"] == movie_id, "genres"].values
    
    new_genre = (
        corresponding_genre[0]
        if len(corresponding_genre) > 0 and not pd.isna(corresponding_genre[0])
        else "Action"
    )
    
    ratings.at[index, "genres"] = new_genre
    
ratings["genres"] = ratings["genres"].str.lower()

d = defaultdict(LabelEncoder)

cols_cat = ["userId", "movieId"]
for c in cols_cat:
    d[c].fit(ratings[c].unique())
    ratings[c] = d[c].transform(ratings[c])

grouped_rating = ratings.groupby("movieId")["genres"].first().reset_index()
n_genres = set("|".join(grouped_rating["genres"]).split("|"))

### Create the one-hot representation of the genres to use them in Matrix Factorization

In [ ]:
# Create a dictionary mapping movie IDs to genre vectors using one-hot encoding
movie_to_genre_vector = {}

# Convert genres column to a list of lists
genres_list = [genres.split("|") for genres in grouped_rating["genres"]]

# Use MultiLabelBinarizer for one-hot encoding
mlb = MultiLabelBinarizer()
genre_vectors = mlb.fit_transform(genres_list)

for idx, row in grouped_rating.iterrows():
    movie_id = row["movieId"]
    genre_vector = torch.FloatTensor(genre_vectors[idx])

    movie_to_genre_vector[movie_id] = genre_vector

train_ratings = ratings[ratings.rating != -1]
test_ratings = ratings[ratings.rating == -1]

train_ratings.reset_index(drop=True, inplace=True)
test_ratings.reset_index(drop=True, inplace=True)

df_train = train_ratings
df_train.head()

### Initialize the MovieDataset and the Dataloaders

In [ ]:
# Create the DataLoader
batch_size = 32

train_dataset = MovieRatingDataset(df_train, movie_to_genre_vector)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = MovieRatingDataset(test_ratings, movie_to_genre_vector)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

### Training Loop: Train the Matrix Factorization model

In [ ]:
# Training loop parameter setting
n_users = len(ratings["userId"].unique())
n_movies = len(ratings["movieId"].unique())
n_genres = len(set("|".join(grouped_rating["genres"]).split("|")))

n_factors = 15

model = MatrixFactorization(n_users, n_movies, n_factors, n_genres)

criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.06)
scheduler = ReduceLROnPlateau(optimizer, verbose=True)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
# Training loop routine
epoch_train_losses, epoch_val_losses, epoch_val_rmse = [], [], []

num_epochs = 7

for epoch in tqdm(range(num_epochs), desc="Training", unit="epoch"):
    train_losses, val_losses = [], []
    predicted_ratings, true_ratings = [], []

    for batch_x, batch_y, batch_genres in train_dataloader:
        # Get the data
        users = batch_x[0].to(device)
        movies = batch_x[1].to(device)

        genres = batch_genres.to(device)
        batch_y = batch_y.to(device, dtype=torch.float)
        
        # Set the gradients to zero
        optimizer.zero_grad()

        # Forward pass
        outputs = model(users, movies, genres)

        # Calculate the loss
        loss = criterion(outputs, batch_y)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Keep track of the loss
        train_losses.append(loss.item())

    # Calculate statistics
    epoch_train_loss = np.mean(train_losses)
    epoch_train_losses.append(epoch_train_loss)

    # Print the results for this epoch:
    s = f"Epoch: {epoch}, Train Loss: {epoch_train_loss:0.3f}, "
    print(s)

print("Training complete!")

At this point, we have a trained Matrix Factorization model and we are ready to generate the predictions!

## Section 3: User Rating Prediction Generation 🔧

Now that we completed the training of the Matrix Factorization model, we generate the predictions to be submitted to Kaggle.

### Generate the predictions by running the testing data.

In [ ]:
# Create a list to store the predicted ratings
predicted_ratings = []

model.eval()
for user_movie_Id, _, genre_vector in test_dataloader:
    # Fetch the data
    user_id = user_movie_Id[0].to(device)
    movie_id = user_movie_Id[1].to(device)
    genre_vector_ = genre_vector.to(device)

    # Generate the rating prediction
    predicted_rating = model(user_id, movie_id, genre_vector_)
    
    # Keep the results
    predicted_ratings.append(predicted_rating.item())

# Add the data to the final dataframe
test_ratings["rating"] = predicted_ratings
test_ratings.reset_index(inplace=True)
test_ratings.rename(columns={"index": "Id"}, inplace=True)

test_ratings.head()

### Save the generated predictions for submission

In [ ]:
# Keep only the columns needed for the submission
test_ratings.drop(columns=["userId", "movieId", "genres"], inplace=True)

# Save the predictions to a csv file
filename = "final_predictions.csv"
test_ratings.to_csv(filename, index=False)

print(f"Predictions saved to {filename}")